#### Reading text

In [1]:
with open("./the-verdict.txt", "r", encoding="utf-8") as file:
    raw_text = file.read()

In [2]:
print(f"Total characters: {len(raw_text)}")

Total characters: 20479


In [3]:
raw_text[:100]

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g'

#### Tokenization

In [4]:
import re
text = "Hello, World. This, is a test."
result = re.split(r'([,.]|\s)', text) # split on spaces, commas and dot.
print(result)

['Hello', ',', '', ' ', 'World', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [5]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'World', '.', 'This', ',', 'is', 'a', 'test', '.']


When developing a simple tokenizer whether we should encode whitesaces as separate characters or not depend on our use case. It is useful in case like Python coding where indentation is important.

In [6]:
# more split characters
import re
text = "Hello, WOrld. Is this-- a test? or what?!"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'WOrld', '.', 'Is', 'this', '--', 'a', 'test', '?', 'or', 'what', '?', '!']


Tokenizing raw text in token

In [7]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]

print(f"Total length of preprocessed text: {len(preprocessed)}")

Total length of preprocessed text: 4690


In [8]:
preprocessed[:10]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius']

#### Converting token in Ids

In [9]:
# creating vocabulary to map token into ids

all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

word_to_id = {word: i for i, word in enumerate(all_words)}
id_to_word = {i: word for word, i in word_to_id.items()}

In [10]:
word_to_id["genius"]

486

In [11]:
for index, token in enumerate(word_to_id.items()):
    print(token)
    if index > 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


In [12]:
print(f"The vocab size: {vocab_size}")

The vocab size: 1130


### A simple Tokenizer

In [13]:
class SimpleTokenizer:
    def __init__(self, vocab: dict):
        self.str_to_int = vocab
        self.int_to_str = {i: w for w, i in vocab.items()}
        
    def encode(self, text: str) -> list:
        preprocessed = re.split(r'([.,?_!"()\']|--|\s)', text)
        preprocessed = [text for text in preprocessed if text.strip()]
        ids = [self.str_to_int[word] for word in preprocessed]
        return ids
    
    def decode(self, ids: list) -> str:
        text = " ".join([self.int_to_str[i] for i in ids])
        # replace space before punctuation
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [14]:
tokenizer = SimpleTokenizer(word_to_id)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [15]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

#### Adding special context tokens

In [16]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {word: i for i, word in enumerate(all_tokens)}

print(f"The new vocab size is: {len(vocab)}")

The new vocab size is: 1132


### Tokenizer Version2

In [17]:
class SimpleTokenizerv2:
    def __init__(self, vocab: dict):
        self.str_to_int = vocab
        self.int_to_str = {i: word for word, i in vocab.items()}
    
    def encode(self, text: str) -> list:
        preprocessed = re.split(r'([.,:;?_!"()\']|--|\s)', text)
        preprocessed = [text for text in preprocessed if text.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[word] for word in preprocessed]
        return ids
    
    def decode(self, ids: list) -> str:
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [18]:
tokenizer = SimpleTokenizerv2(vocab)

text1 = "It's the last he painted, you know, Mrs. Gisburn said with pardonable pride."
text2 = "In the end, we will remember not the words of our enemies, but the silence of our friends."

text = " <|endoftext|> ".join((text1, text2))

print(text)

It's the last he painted, you know, Mrs. Gisburn said with pardonable pride. <|endoftext|> In the end, we will remember not the words of our enemies, but the silence of our friends.


In [19]:
tokenizer.encode(text)

[56,
 2,
 850,
 988,
 602,
 533,
 746,
 5,
 1126,
 596,
 5,
 67,
 7,
 38,
 851,
 1108,
 754,
 793,
 7,
 1130,
 55,
 988,
 388,
 5,
 1131,
 1131,
 832,
 711,
 988,
 1131,
 722,
 736,
 1131,
 5,
 239,
 988,
 1131,
 722,
 736,
 1131,
 7]

In [20]:
tokenizer.encode("<|unk|> <|unk|> <|unk|> <|unk|>")

[1131, 1131, 1131, 1131]

In [21]:
tokenizer.encode("<|endoftext|> <|unk|> <|unk|> <|endoftext|>")

[1130, 1131, 1131, 1130]

In [22]:
tokenizer.decode(tokenizer.encode(text))

"It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride. <|endoftext|> In the end, <|unk|> <|unk|> remember not the <|unk|> of our <|unk|>, but the <|unk|> of our <|unk|>."

## BytePair Encoding
The BytePair Encoding was used to train GPT model which can handle OOV words more than our tokenizer

In [23]:
import tiktoken

print(tiktoken.__dict__)

{'__name__': 'tiktoken', '__doc__': None, '__package__': 'tiktoken', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x000002673DC8BC80>, '__spec__': ModuleSpec(name='tiktoken', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000002673DC8BC80>, origin='C:\\Users\\acer\\AppData\\Roaming\\Python\\Python312\\site-packages\\tiktoken\\__init__.py', submodule_search_locations=['C:\\Users\\acer\\AppData\\Roaming\\Python\\Python312\\site-packages\\tiktoken']), '__path__': ['C:\\Users\\acer\\AppData\\Roaming\\Python\\Python312\\site-packages\\tiktoken'], '__file__': 'C:\\Users\\acer\\AppData\\Roaming\\Python\\Python312\\site-packages\\tiktoken\\__init__.py', '__cached__': 'C:\\Users\\acer\\AppData\\Roaming\\Python\\Python312\\site-packages\\tiktoken\\__pycache__\\__init__.cpython-312.pyc', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nide

In [24]:
tokenizer = tiktoken.get_encoding("gpt2")

In [25]:
tokenizer

<Encoding 'gpt2'>

Here we dont need UNK token because GPT uses Byte pair encoding which takes subwords to tokenize.

In [26]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [27]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


#### Data Sampling with Sliding Window

In [28]:
with open("./the-verdict.txt", "r", encoding="utf-8") as file:
    raw_text = file.read()
    
enc_text = tokenizer.encode(raw_text)

print(len(enc_text))

5145


In [29]:
tokenizer.decode([1,2])

'"#'

we want to create input and target pairs such that the target is one position to the right

In [30]:
enc_sample = enc_text[100:300]

In [31]:
enc_sample[:10]

[5469, 438, 14363, 938, 4842, 1650, 353, 438, 2934, 489]

In [32]:
CONTEXT_SIZE = 4

x = enc_sample[:CONTEXT_SIZE]
y = enc_sample[1:CONTEXT_SIZE + 1]

print(f"Context: {x}")
print(f"Target:         {y}")

Context: [5469, 438, 14363, 938]
Target:         [438, 14363, 938, 4842]


In [33]:
for i in range(1, CONTEXT_SIZE + 1):
    context = enc_sample[:i]
    target = enc_sample[i]
    
    print(f"Context: {context} Target: {target}")

Context: [5469] Target: 438
Context: [5469, 438] Target: 14363
Context: [5469, 438, 14363] Target: 938
Context: [5469, 438, 14363, 938] Target: 4842


In [34]:
for i in range(1, CONTEXT_SIZE + 1):
    context = enc_sample[:i]
    target = enc_sample[i]
    
    print(f"Context: {tokenizer.decode(context)} Target: {tokenizer.decode([target])}")

Context: wing Target: --
Context: wing-- Target: his
Context: wing--his Target:  last
Context: wing--his last Target:  Chicago


#### Creating Context and Target Pair for our Dataset

```python
x = [["My", "name", "is", "Alex"],
    ["Bob", "I", "live", "in"]]

y = [["name", "is", "Alex", "Bob"],
     ["I", "live", "in", "Antartica"]]
```

In [35]:
import torch

In [36]:
from torch.utils.data import Dataset, DataLoader

class DatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, stride):
        """create a dataset from the txt"""
        self.input_ids = []
        self.target_ids = []
        
        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
        
        for i in range(0, len(token_ids) - max_length, stride): # stride is for overlapping
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
            return len(self.input_ids)
        
    def __getitem__(self, idx):
            return self.input_ids[idx], self.target_ids[idx]

In [37]:
def create_dataloaderV1(text, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    # initializing tokenizer
    tokenier = tiktoken.get_encoding("gpt2")

    dataset = DatasetV1(text, tokenizer, max_length, stride)
    
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size, shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    
    return dataloader

#### creating context pair dataset

In [38]:
create_dataloaderV1

<function __main__.create_dataloaderV1(text, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0)>

In [39]:
with open("./the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [40]:
dataloader = create_dataloaderV1(
    raw_text,
    batch_size=1,
    max_length=4,
    stride=1,
    shuffle=False
)

In [41]:
data_iter = iter(dataloader)

first_batch = next(data_iter)

first_batch

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]

In [42]:
next(data_iter)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]

In [43]:
dataloader = create_dataloaderV1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


#### Creating token embeddings

In [44]:
# sample embedding
import torch

sample_vocab_size = 6
sample_output_dim = 3

embedding_layer = torch.nn.Embedding(sample_vocab_size, sample_output_dim)

print(embedding_layer.weight)

Parameter containing:
tensor([[ 1.0213, -1.2859, -0.4639],
        [ 0.7733,  0.9078,  0.8196],
        [ 0.5637, -0.5675, -0.7319],
        [ 0.9674, -1.3192,  2.6806],
        [ 0.6213, -0.0943,  0.3628],
        [ 0.3000, -0.1667, -0.1972]], requires_grad=True)


In [45]:
embedding_layer(torch.tensor([0])) # will get the embedding vector for the token 0

tensor([[ 1.0213, -1.2859, -0.4639]], grad_fn=<EmbeddingBackward0>)

### Encoding word position

In [47]:
tokenizer.n_vocab

50257

In [48]:
VOCAB_SIZE = tokenizer.n_vocab
EMBEDDING_DIM = 256

In [49]:
token_embedding_layer = torch.nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)

For our dataset with batch size of 8 and token of 4 each we will get 8 * 4 * 256 matrix

In [50]:
max_length = 4
dataloader = create_dataloaderV1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [51]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [52]:
CONTEXT_LENGTH = max_length

pos_embedding_layer = torch.nn.Embedding(CONTEXT_LENGTH, EMBEDDING_DIM)

In [55]:
pos_embeddings = pos_embedding_layer(torch.arange(CONTEXT_LENGTH)) # absolute position encoding

pos_embeddings.shape

torch.Size([4, 256])

In [56]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
